In [5]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS from flask_cors
# import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
import pymysql
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

app = Flask(__name__)
CORS(app) 

# # MySQL configuration
# db = pymysql.connect(
#     host='109.106.254.1',
#     user='u690159757_kim',
#     password='*yKa&T6y4#sJ8XvE9j',
#     db='u690159757_kim',
#     cursorclass=pymysql.cursors.DictCursor  
# )
# MySQL configuration
db = pymysql.connect(
    host='mysql5049.site4now.net',
    user='aa0682_movies',
    password='Password1234.',
    db='db_aa0682_movies',
    connect_timeout=8800,
    cursorclass=pymysql.cursors.DictCursor
     
)

In [69]:
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute(''' SELECT keyword FROM article''')
data = cursor.fetchall()
db.commit()
cursor.close()




In [70]:
data = [list(item.values())[0] for item in data]


In [67]:
uniquwdata= np.unique(data)

In [76]:
data = ",".join(data)
data = data.split()

In [77]:
data

['G,e,n,d,e,r,,',
 ',g,e,n,d,e,r',
 ',e,d,u,c,a,t,i,o,n,,',
 ',g,e,n,d,e,r',
 ',p,e,r,s,p,e,c,t,i,v,e,s,,',
 ',e,x,p,l,o,r,a,t,o,r,y',
 ',s,t,u,d,y,,',
 ',P,h,i,l,i,p,p,i,n,e',
 ',E,d,u,c,a,t,i,o,n',
 ',S,y,s,t,e,m,,',
 ',T,h,e,o,r,i,e,s',
 ',o,f',
 ',L,e,a,r,n,i,n,g,,',
 ',S,D,G,,',
 ',Q,u,e,z,o,n',
 ',C,i,t,y',
 ',U,n,i,v,e,r,s,i,t,y,,,c,a,l,l',
 ',c,e,n,t,e,r',
 ',r,e,p,r,e,s,e,n,t,a,t,i,v,e,s,,',
 ',l,a,b,o,r',
 ',p,r,o,d,u,c,t,i,v,i,t,y,,',
 ',a,n,d',
 ',e,m,p,l,o,y,m,e,n,t,,,s,o,c,i,a,l',
 ',i,s,o,l,a,t,i,o,n,,',
 ',w,o,r,k',
 ',s,a,t,i,s,f,a,c,t,i,o,n,,',
 ',s,t,r,e,s,s,,',
 ',w,o,r,k',
 ',p,r,o,d,u,c,t,i,v,i,t,y,,,e,l,e,c,t,r,o,n,i,c',
 ',w,a,l,l,e,t,,',
 ',a,c,c,u,r,a,c,y',
 ',o,f',
 ',p,a,y,m,e,n,t,,',
 ',o,n,-,t,i,m,e',
 ',d,e,l,i,v,e,r,y,,',
 ',a,c,c,e,s,s,i,b,i,l,i,t,y',
 ',a,n,d',
 ',e,f,f,i,c,i,e,n,c,y,,',
 ',s,e,c,u,r,i,t,y',
 ',a,n,d',
 ',r,e,l,i,a,b,i,l,i,t,y,,,S,a,f,e,t,y',
 ',o,f',
 ',l,i,f,e',
 ',a,t',
 ',s,e,a',
 ',p,r,o,g,r,a,m',
 ',(,S,O,L,A,S,),,',
 ',s,e,a,f,a

In [30]:
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute('SELECT * FROM article')
data = cursor.fetchall()

id = [row['article_id'] for row in data]
# overviews_orig = [row['abstract'] for row in data]
overviews = [row['abstract'] for row in data]
titles = [row['title'] for row in data] 


# Preprocess the movie overviews
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

for n, name in enumerate(overviews):
    temp = name.lower().split(" ")
    temp = [''.join([letter for letter in word if letter.isalnum()]) for word in temp]
    temp = [word for word in temp if word not in stop_words]
    temp = ' '.join(temp)
    overviews[n] = temp

# Calculate cosine similarity
    from sklearn.feature_extraction.text import CountVectorizer

    vectorizer = TfidfVectorizer().fit_transform(overviews)
    cosine_sim = cosine_similarity(vectorizer)

# Function to get movie recommendations
def get_movie_recommendations( movie_title, similarity_matrix):
    if type(movie_title) == str:
        movie_idx = titles.index(movie_title)
        
    else: 
        movie_idx = movie_title
        print(movie_title)
    similar_movies = list(enumerate(similarity_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    
    # Calculate a recommendation score based on similarity (cosine similarity)
    for i in similar_movies:
        if(i[1]< 0.15):
            break
        recommended_movie_title = titles[i[0]]
        movie_description = overviews[i[0]]
        recommended_movies.append({'title': recommended_movie_title, 'description': movie_description,'movie_id': id[i[0]], 'score':i[1]})

    return recommended_movies

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kimbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
get_movie_recommendations("Online Shopping Experience of UDM Faculty Members in Manila's Market Place",cosine_sim)

[{'title': "Online Shopping Experience of UDM Faculty Members in Manila's Market Place",
  'description': 'universidad de manila udm formerly known city college manila established 1995 city council manilas ordinance 7885 25th founding anniversary april 2020 udm implemented repositioning strategy adhere original mission preeminent science technology higher education institution manila education 40 focal point core course offerings universidad de manila portal designed current prospective students faculty universidad de manila portal ena bled faculty members perform variety academicrelated tasks including viewing course ad mission listsstatus academic calendar lectures gradesperformance beyond class hours portal facilitates communication professors students study initi ative usability analysis determine needs udm community expand university information communications technology offices service offerings icto study deter mined faculty portals usability terms features frequency use serves 

In [88]:
pip install gensim


   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB 991.0 kB/s eta 0:00:25
   ---------------------------------------- 0.1/24.0 MB 1.3 MB/s eta 0:00:19
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:21
   ---------------------------------------- 0.2/24.0 MB 958.1 kB/s eta 0:00:25
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:22
    --------------------------------------- 0.4/24.0 MB 1.3 MB/s eta 0:00:19
    --------------------------------------- 0.4/24.0 MB 1.3 MB/s eta 0:00:18
    --------------------------------------- 0.5/24.0 MB 1.4 MB/s eta 0:00:17
    --------------------------------------- 0.6/24.0 MB 1.4 MB/s eta 0:00:17
   - -------------------------------------- 0.7/24.0 MB 1.5 MB/s eta 0:00:16
   - -------------------------------------- 0.8/24.0 MB 1.6 MB/s eta 0:00:15
   - -------------------------------------- 0.9/24.0 MB 1.7 MB/s eta 0:00:14
  

Most Similar Subject: Communications Infrastructure and Facilities


In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
article_title = "A Classroom-based Action Research on Selected First Year Infor- mation Technology Students of Quezon City University for the Sec- ond Semester of Academic Year 2021-2022"
article_abstract = "This action research reports the results of a classroom-based study conducted in general education courses to examine whether the integration of video recordings would contribute to the improvement of the oral communication skills of students. It also aims to find out what are the perceptions of students using video recording their online learning. twenty students enrolled in general courses participated in this study. To determine the effectiveness of video recording on the students' oral communication skills, the speaking test of IELTS was modified as pre- and post-test. In addition to the analysis of the scores assigned to the students by two raters using Cohen's kappa, an analysis of students' speaking in pre- and post-test was undertaken to determine whether measures of students' oral communication output differ as a result of the video recording. The results suggest that the incorporation of video recordings into the speaking activities did not have significant improvement in their oral communication skills. However, the analysis of the qualitative data showed that the students were able to apply those communication skills in speaking, and they have gained confidence and insights on how to improve their skills through the comments given by the researchers."

# List of subject areas or journals with corresponding content
subject_areas = [
    "Information Technology",
    "Industrial Engineering",
    "Electronics",
    "Management Information Systems",
    "Cybersecurity and Privacy",
    "Communications Infrastructure and Facilities",
    "New Energy Technology and Energy-saving Technology",
    "Power Generation",
    "Accountancy",
    "Business Administration",
    "Criminology",
    "Economics",
    "Entrepreneurship",
    "Gender Studies",
    "Local Governance and Administration",
    "Public Administration",
    "Political Science",
    "Social Policy",
    "Social Work",
    "Sociology",
    "Educational Reforms and Policy Studies",
    "Academe-Industry Partnership",
    "Alternative Learning System",
    "Open Education",
    "Distance Learning",
    "Outcome-based Education",
    "Faculty Development",
    "Educational Technology",
    "Technical Vocation"]

# Preprocess and vectorize the article
tfidf_vectorizer = TfidfVectorizer()
article_vector = tfidf_vectorizer.fit_transform([article_title + " " + article_abstract])

# Calculate cosine similarity with each subject area
similarities = [cosine_similarity(article_vector, tfidf_vectorizer.transform([subject_area]))[0][0] for subject_area in subject_areas]

# Determine the most similar subject area
most_similar_subject_area = subject_areas[similarities.index(max(similarities))]

print("Most Similar Subject Area:", most_similar_subject_area)


Most Similar Subject Area: Cybersecurity and Privacy


In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute('SELECT * FROM article')
data = cursor.fetchall()

# Assuming 'title' is the column name in your database
titles = [row['title'] for row in data]
print(titles)

['Addressing The Trade Offs Between Labor Productivity And Employment Among Call Center Agents In Metro Manila', 'The Effects of Social Isolation, Remote Work Satisfaction, and Stress on Perceived Remote Work Productivity of Business Process Outsourcing Employees in National Capital Region: A Gender Difference Approach', 'The Effectiveness of Electronic Wallet as alternative for cash transaction for Quezon City Entrepreneurship Students', "Seafarers' Safety of Life at Sea (SOLAS) and Skills Performance A Basis for Risk Reduction Management Model", "Online Shopping Experience of UDM Faculty Members in Manila's Market Place", 'G Sa Ugbo: Food Safety and Sanitation Practices of Food Establishments in Tondo, Manila', 'Responsiveness of Financial Institutions to Tourism-Related Business in Puerto Princesa City', 'A Classroom-based Action Research on Selected First Year Infor- mation Technology Students of Quezon City University for the Sec- ond Semester of Academic Year 2021-2022', 'Online 

In [112]:
pip install spacy



     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     -------------------------------------- 45.9/45.9 kB 774.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/64.1 kB ? eta -:--:--
     -------------------------------------- - 61.4/64.1 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 64.1/64.1 kB 855.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/181.6 kB ? eta -:--:--
     --------------- ----------------------- 71.7/181.6 kB 4.1 MB/s eta 0:00:01
     ------------------------------------ - 174.1/181.6 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 181.6/181.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.2/12.1 MB 5.0 MB/s eta 0:00:03
   - -------------------------------------- 0.3/12.1 MB 3.8 MB/s eta 0:00:04
   - -------------------------------------- 0.5/12.1 MB 3.9 MB/s eta 0:00:04
 

In [115]:
!python -m spacy download en_core_web_md


     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 640.0 kB/s eta 0:01:07
     --------------------------------------- 0.0/42.8 MB 262.6 kB/s eta 0:02:43
     --------------------------------------- 0.1/42.8 MB 290.5 kB/s eta 0:02:28
     --------------------------------------- 0.1/42.8 MB 357.2 kB/s eta 0:02:00
     --------------------------------------- 0.1/42.8 MB 514.3 kB/s eta 0:01:23
     --------------------------------------- 0.2/42.8 MB 655.4 kB/s eta 0:01:06
     --------------------------------------- 0.2/42.8 MB 655.1 kB/s eta 0:01:05
     --------------------------------------- 0.2/42.8 MB 654.9 kB/s eta 0:01:05
     --------------------------------------- 0.3/42.8 MB 680.9 kB/s eta 0:01:03
     --------------------------------------- 0.3/42.8 MB 655.2 k

In [9]:
import spacy
import numpy as np

# Load spaCy model with word vectors (you may need to download the model)
nlp = spacy.load("en_core_web_md")

# Function to calculate cosine similarity between two vectors
def cosine_similarity_vec(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Example usage
article_title = "this is about education"
article_abstract = " inclusion gender education education education general education curriculum acknowledges importance developing learners' intellectual personal civil competencies first semester School Year 2021 2022, Quezon City University offered Gender  Society GEE1 first time elective course for freshmen students"
article_text = article_abstract.lower()

# Process the article text with spaCy
article_doc = nlp(article_text)

# List of subjects
subjects = [ "Social Science", "Education","Science and Technology"]

# Calculate cosine similarity between the article vector and each subject vector
similarities = [cosine_similarity_vec(article_doc.vector, nlp(subject).vector) for subject in subjects]

# Determine the most similar subject
most_similar_subject = subjects[np.argmax(similarities)]

print("Most Similar Subject:", most_similar_subject)


Most Similar Subject: Science and Technology


In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute('SELECT * FROM article')
data = cursor.fetchall()

# Assuming 'title' is the column name in your database
titles = [row['title'] for row in data]


# Modify the labeling logic to check if "Online Shopping" is present in the title
y = np.array([1 if "Online Learning" in title else 0 for title in titles])

# Experiment with vectorization parameters
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), min_df=1, max_df=0.8)  # Adjust parameters
X = vectorizer.fit_transform(titles)
# Manually inspect titles that are expected to be labeled as "Online Shopping" but are not
mismatched_titles = [title for title, label in zip(titles, y) if label == 0 and "Online Learning" in title]

# Print mismatched titles
print("Mismatched Titles:")
for title in mismatched_titles:
    print(title)

# Debugging prints
print("Number of titles:", len(titles))
print("Number of positive labels (Online Shopping):", np.sum(y))
print("Vectorized features shape:", X.shape)
print("Vocabulary:", vectorizer.get_feature_names_out()[:10])  # Print first 10 features

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X, y)

def search_topic(user_query):
    user_query_vectorized = vectorizer.transform([user_query])
    prediction = clf.predict(user_query_vectorized)

    # Debugging print
    print("Prediction:", prediction)

    # Check if the prediction array has size 1
    if prediction.size == 1:
        # Directly compare the value with the target
        is_relevant = prediction[0] == 1
        relevant_articles = [data[0]] if is_relevant else []
    else:
        # Use the original indexing logic
        relevant_articles = [data[i] for i in range(len(data)) if prediction[i] == 1]

    return {'results': relevant_articles}

# Example usage
search_result = search_topic("online learning")
print(search_result)


Mismatched Titles:
Number of titles: 189
Number of positive labels (Online Shopping): 5
Vectorized features shape: (189, 2265)
Vocabulary: ['19' '19 pandemic' '19 recovered' '2014' '2014 2019' '2019' '2021'
 '2021 2022' '2022' '2022 philippine']
Prediction: [0]
{'results': []}


In [25]:
db.ping(reconnect=True) 
with db.cursor() as cursor:
    cursor.execute('SELECT * FROM watchedMovies')
    data = cursor.fetchall()[::-1]
    movie_ids = [row['movie_id'] for row in data]
    movie_ids = np.unique(movie_ids)
    # print(data[0]['movie_id'],"ddata")
    temp=[]
    results=[]
    for i in range(len(movie_ids)):
            recommendations = get_movie_recommendations(i, cosine_sim)[1:]
            if len(recommendations) < 1: continue
            temp.append(recommendations)
            if len(temp) > 5: break
        #  print(data[i]['movie_id'])
    
    # results =[row['movie_id'] for row in temp]
    print('-----',temp,'---------------')
    
    for movie_group in temp:
        for movie in movie_group:
            # Access movie information
            
            movie_id = movie['movie_id']
            results.append(movie_id)
    results = np.unique(results)
    print(results)

0


TypeError: 'int' object is not subscriptable